[日本語Wikipedia](https://ja.wikipedia.org/)の「[人工知能](https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD)」に関する記事からテキスト部分を抜き出したファイルが[ai.ja.zip](https://nlp100.github.io/data/ai.ja.zip)に収録されている． この文章を[CaboCha](https://taku910.github.io/cabocha/)や[KNP](http://nlp.ist.i.kyoto-u.ac.jp/index.php?KNP)等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

In [26]:
import CaboCha

cp = CaboCha.Parser()
with open('ai.ja.txt', encoding='utf-8') as f:
    rows = f.read().split('\n')

with open('ai.ja.txt.parsed', 'w', encoding='utf-8') as f:
    for row in rows:
        if row and row != '人工知能':
            for sentence in row.strip().split('。'):
                if sentence:
                    tree = cp.parse(sentence + '。')
                    f.write(tree.toString(CaboCha.FORMAT_LATTICE))

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [1]:
from dataclasses import dataclass
from typing import List

@dataclass
class Morph:
    surface: str
    base: str
    pos: str
    pos1: str

with open('ai.ja.txt.parsed', encoding='utf-8') as f:
    morphs_list: List[List[Morph]] = []
    for rows in f.read().split('EOS'):
        morphs: List[Morph] = []
        if rows:
            for row in rows.split('\n'):
                if row and not row.startswith('*'):
                    # 表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
                    surface, feature = row.split('\t')
                    # len(feature)==7のものも存在する模様
                    pos, pos1, _, _, _, _, base, *_ = feature.split(',')
                    morphs.append(Morph(surface=surface, base=base, pos=pos, pos1=pos1))
        if morphs:
            morphs_list.append(morphs)
            
morphs_list[-1]

[Morph(surface='おかげ', base='おかげ', pos='名詞', pos1='一般'),
 Morph(surface='で', base='で', pos='助詞', pos1='格助詞'),
 Morph(surface='、', base='、', pos='記号', pos1='読点'),
 Morph(surface='意見', base='意見', pos='名詞', pos1='サ変接続'),
 Morph(surface='の', base='の', pos='助詞', pos1='連体化'),
 Morph(surface='違い', base='違い', pos='名詞', pos1='ナイ形容詞語幹'),
 Morph(surface='は', base='は', pos='助詞', pos1='係助詞'),
 Morph(surface='明らか', base='明らか', pos='名詞', pos1='形容動詞語幹'),
 Morph(surface='に', base='に', pos='助詞', pos1='格助詞'),
 Morph(surface='なっ', base='なる', pos='動詞', pos1='自立'),
 Morph(surface='た', base='た', pos='助動詞', pos1='*'),
 Morph(surface='と', base='と', pos='助詞', pos1='格助詞'),
 Morph(surface='は', base='は', pos='助詞', pos1='係助詞'),
 Morph(surface='思い', base='思う', pos='動詞', pos1='自立'),
 Morph(surface='ます', base='ます', pos='助動詞', pos1='*'),
 Morph(surface='が', base='が', pos='助詞', pos1='接続助詞'),
 Morph(surface='、', base='、', pos='記号', pos1='読点'),
 Morph(surface='果たして', base='果たして', pos='副詞', pos1='一般'),
 Morph(surface='何', b

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [15]:
from collections import defaultdict
from dataclasses import dataclass
from typing import DefaultDict, List

@dataclass
class Chunk:
    morphs: List[Morph]
    dst: int
    srcs: List[int]
            
with open('ai.ja.txt.parsed', encoding='utf-8') as f:
    chunks_list: List[List[Chunk]] = []
    for rows in f.read().split('EOS'):
        morphs_list: List[List[Morph]] = []
        dsts: List[int] = []
        chunk_num2srcs: DefaultDict[int, List[int]] = defaultdict(list)
        if rows:
            for row in rows.split('\n'):
                if row:
                    if row.startswith('*'):
                        # * 文節番号 かかり先の文節番号 主辞/機能語の位置 係り関係のスコア
                        chunk_num = int(row.split(' ')[1])
                        dst = int(row.split(' ')[2][:-1])
                        chunk_num2srcs[dst].append(chunk_num)
                        dsts.append(dst)
                        morphs_list.append([])
                    else:
                        # 表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
                        surface, feature = row.split('\t')
                        # len(feature)==7のものも存在する模様
                        pos, pos1, _, _, _, _, base, *_ = feature.split(',')
                        morphs_list[-1].append(Morph(surface=surface, base=base, pos=pos, pos1=pos1))

        if morphs_list:
            chunks_list.append([
                Chunk(morphs=morphs, dst=dst, srcs=chunk_num2srcs.get(i, []))
                for i, (morphs, dst) in enumerate(zip(morphs_list, dsts))
            ])

for i, chunk in enumerate(chunks_list[-1]):
    print(f'Chunk {i}')
    print(f'  dst: {chunk.dst}')
    print(f'  srcs: {chunk.srcs}')
    for morph in chunk.morphs:
        print(f'  morph: {morph}')

Chunk 0
  dst: 3
  srcs: []
  morph: Morph(surface='おかげ', base='おかげ', pos='名詞', pos1='一般')
  morph: Morph(surface='で', base='で', pos='助詞', pos1='格助詞')
  morph: Morph(surface='、', base='、', pos='記号', pos1='読点')
Chunk 1
  dst: 2
  srcs: []
  morph: Morph(surface='意見', base='意見', pos='名詞', pos1='サ変接続')
  morph: Morph(surface='の', base='の', pos='助詞', pos1='連体化')
Chunk 2
  dst: 3
  srcs: [1]
  morph: Morph(surface='違い', base='違い', pos='名詞', pos1='ナイ形容詞語幹')
  morph: Morph(surface='は', base='は', pos='助詞', pos1='係助詞')
Chunk 3
  dst: 4
  srcs: [0, 2]
  morph: Morph(surface='明らか', base='明らか', pos='名詞', pos1='形容動詞語幹')
  morph: Morph(surface='に', base='に', pos='助詞', pos1='格助詞')
  morph: Morph(surface='なっ', base='なる', pos='動詞', pos1='自立')
  morph: Morph(surface='た', base='た', pos='助動詞', pos1='*')
  morph: Morph(surface='と', base='と', pos='助詞', pos1='格助詞')
  morph: Morph(surface='は', base='は', pos='助詞', pos1='係助詞')
Chunk 4
  dst: 8
  srcs: [3]
  morph: Morph(surface='思い', base='思う', pos='動詞', pos1='

## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

chunk: おかげで - sep: 明らかになったとは
chunk: 意見の - sep: 違いは
chunk: 違いは - sep: 意見の	明らかになったとは
chunk: 明らかになったとは - sep: おかげで	違いは	思いますが
chunk: 思いますが - sep: 明らかになったとは	つくのでしょうかと
chunk: 果たして - sep: つくのでしょうかと
chunk: 何か - sep: つくのでしょうかと
chunk: 決着が - sep: つくのでしょうかと
chunk: つくのでしょうかと - sep: 思いますが	果たして	何か	決着が	発言し
chunk: 発言し - sep: つくのでしょうかと	答えている
chunk: 伊勢田は - sep: 答えている
chunk: 決着は - sep: つかないでしょうねと
chunk: つかないでしょうねと - sep: 決着は	答えている
chunk: 答えている - sep: 発言し	伊勢田は	つかないでしょうねと


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，[Graphviz](http://www.graphviz.org/)等を用いるとよい．

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．
- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
```
作り出す	で は を
```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
- コーパス中で頻出する述語と格パターンの組み合わせ
- 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．
- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
```
作り出す	で は を	会議で ジョンマッカーシーは 用語を
```

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．
- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．
```
学習を行う	に を	元に 経験を
```

## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．
- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した
```

KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した
```

## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．
- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．
- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
```

KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを
```